In [193]:
import polars as pl
from constants import *
pl.Config.set_auto_structify(True)

polars.config.Config

In [261]:
trail_df = pl.read_json('trial_data.json')



In [262]:
# trail_df.schema
pl.Config.allow_object = True
trail_df = trail_df.with_columns(
    pl.col('trial_outcome').cast(pl.Utf8).replace(OUTCOME_MAP).alias('outcome_label'))
trail_df

trial_id,trial_outcome,prisoner_0,prisoner_1,prisoners,outcome_label
i64,i64,struct[4],struct[4],list[struct[4]],str
1,1,"{0,0,1,0.6}","{1,3,0,0.6}","[{0,0,1,0.6}, {1,3,0,0.6}]","""A Rat B"""
2,2,"{0,0,0,0.93}","{1,0,1,0.56}","[{0,0,0,0.93}, {1,0,1,0.56}]","""B Rat A"""
3,3,"{0,2,1,0.5}","{1,2,1,0.79}","[{0,2,1,0.5}, {1,2,1,0.79}]","""Both Against"""
4,2,"{0,0,0,1.12}","{1,0,1,1.21}","[{0,0,0,1.12}, {1,0,1,1.21}]","""B Rat A"""
5,0,"{0,1,0,0.6}","{1,1,0,0.6}","[{0,1,0,0.6}, {1,1,0,0.6}]","""Silence"""
…,…,…,…,…,…
995,1,"{0,0,1,0.6}","{1,3,0,0.6}","[{0,0,1,0.6}, {1,3,0,0.6}]","""A Rat B"""
996,2,"{0,0,0,1.22}","{1,0,1,1.12}","[{0,0,0,1.22}, {1,0,1,1.12}]","""B Rat A"""
997,3,"{0,2,1,1.08}","{1,2,1,1.21}","[{0,2,1,1.08}, {1,2,1,1.21}]","""Both Against"""


In [301]:
trail_df.group_by("trial_outcome").agg(
    pl.col("outcome_label").count()).plot.box()

:BoxWhisker   (trial_outcome)

In [197]:
# trail_df.select([pl.col('trial_outcome')]).plot.box() 
trail_df.select([ pl.col('trial_outcome'),pl.col('trial_id') ]).plot.bar(x='trial_outcome', y='trial_id')

:Bars   [trial_outcome]   (trial_id)

In [198]:
trail_df.schema

Schema([('trial_id', Int64),
        ('trial_outcome', Int64),
        ('prisoner_0',
         Struct({'prisoner_id': Int64, 'sentence': Int64, 'choice': Int64, 'morality': Float64})),
        ('prisoner_1',
         Struct({'prisoner_id': Int64, 'sentence': Int64, 'choice': Int64, 'morality': Float64})),
        ('prisoners',
         List(Struct({'prisoner_id': Int64, 'sentence': Int64, 'choice': Int64, 'morality': Float64})))])

In [199]:
trail_df['prisoners'][0][0]['choice']

1

In [200]:
for row in trail_df['prisoners'][0]:
    print(row)
    for x in row:
        print(x)
        print(row[x])

{'prisoner_id': 0, 'sentence': 0, 'choice': 1, 'morality': 0.6}
prisoner_id
0
sentence
0
choice
1
morality
0.6
{'prisoner_id': 1, 'sentence': 3, 'choice': 0, 'morality': 0.6}
prisoner_id
1
sentence
3
choice
0
morality
0.6


In [201]:
pl.DataFrame([prisoner['morality'] for prisoner in trail_df['prisoners'][0]])

column_0
f64
0.6
0.6


In [202]:
out['prisoners'][0][0]

{'prisoner_id': 0, 'sentence': 0, 'choice': 1, 'morality': 0.6}

trial_id,trial_outcome,prisoner_0,prisoner_1,prisoners
i64,str,struct[4],struct[4],list[struct[4]]
1,"""A Rat B""","{0,0,1,0.6}","{1,3,0,0.6}","[{0,0,1,0.6}, {1,3,0,0.6}]"
2,"""B Rat A""","{0,0,0,0.93}","{1,0,1,0.56}","[{0,0,0,0.93}, {1,0,1,0.56}]"
3,"""Both Against""","{0,2,1,0.5}","{1,2,1,0.79}","[{0,2,1,0.5}, {1,2,1,0.79}]"
4,"""B Rat A""","{0,0,0,1.12}","{1,0,1,1.21}","[{0,0,0,1.12}, {1,0,1,1.21}]"
5,"""Silence""","{0,1,0,0.6}","{1,1,0,0.6}","[{0,1,0,0.6}, {1,1,0,0.6}]"
…,…,…,…,…
995,"""A Rat B""","{0,0,1,0.6}","{1,3,0,0.6}","[{0,0,1,0.6}, {1,3,0,0.6}]"
996,"""B Rat A""","{0,0,0,1.22}","{1,0,1,1.12}","[{0,0,0,1.22}, {1,0,1,1.12}]"
997,"""Both Against""","{0,2,1,1.08}","{1,2,1,1.21}","[{0,2,1,1.08}, {1,2,1,1.21}]"
